In [1]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models


In [2]:

cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]

bas_directory = code_directory / "notebooks" / "Bas"
gonem_directory = code_directory / "notebooks" / "Gonem"
# data_file = bas_directory / "cadeautjevoorGonemenLiza.xlsx"
data_file = gonem_directory / "MAIZE_FILTERED_2023-02-25_19-36-41.xlsx"
data_file


df = pd.read_excel(data_file, header=[0, 1], index_col=0)
df.head(5)

AVG_TAVG                                                  \
PARTNER_Labels      Brazil      France     Germany     Hungary     Ukraine   
TIME_PERIOD                                                                  
2005-01-01      269.701032   50.081198   28.651592   -1.306452    0.869290   
2005-02-01      266.743474   30.689158   -1.962406  -25.371429  -38.376438   
2005-03-01      266.528149   76.202630   45.730051   38.383871   -3.794989   
2005-04-01      253.719444  108.081105  101.859649  112.460000   90.622214   
2005-05-01      247.859946  147.068032  134.667233  162.993548  161.493475   

               Corn Price Futures                                  ...  \
PARTNER_Labels             Brazil  France Germany Hungary Ukraine  ...   
TIME_PERIOD                                                        ...   
2005-01-01                 204.50  204.50  204.50  204.50  204.50  ...   
2005-02-01                 222.75  222.75  222.75  222.75  222.75  ...   
2005-03-01                 221.00  221.00  221.00  221.00  221.00  ...   
2005-04-01                 213.50  213.50  213.50  213.50  213.50  ...   
2005-05-01                 230.75  230.75  230.75  230.75  230.75  ...   

               renewable_energy_consumption_perc_of_total                      \
PARTNER_Labels                                     Brazil    France   Germany   
TIME_PERIOD                                                                     
2005-01-01                                         46.130  8.660000  7.280000   
2005-02-01                                         46.175  8.648333  7.389167   
2005-03-01                                         46.220  8.636667  7.498333   
2005-04-01                                         46.265  8.625000  7.607500   
2005-05-01                                         46.310  8.613333  7.716667   

                               unemployment_total                       \
PARTNER_Labels Hungary Ukraine             Brazil    France    Germany   
TIME_PERIOD                                                              
2005-01-01      7.2900    1.27          10.550000  8.880000  11.170000   
2005-02-01      7.3375    1.31          10.478333  8.875833  11.093333   
2005-03-01      7.3850    1.35          10.406667  8.871667  11.016667   
2005-04-01      7.4325    1.39          10.335000  8.867500  10.940000   
2005-05-01      7.4800    1.43          10.263333  8.863333  10.863333   

                                  
PARTNER_Labels Hungary   Ukraine  
TIME_PERIOD                       
2005-01-01       7.190  7.180000  
2005-02-01       7.215  7.149167  
2005-03-01       7.240  7.118333  
2005-04-01       7.265  7.087500  
2005-05-01       7.290  7.056667  

[5 rows x 85 columns]

In [3]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Brazil'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Hungary'),
 ('price', 'Ukraine')]

In [4]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(log).add(std).add(stl)

In [5]:
from windower import WindowGenerator

width = 24
label_width = 6
shift = 6

w = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    train_begin=0, train_end=.97, val_begin=None, val_end=None,
                    test_begin=None, test_end=1.0, connect=True, remove_labels=True, label_columns=label_columns)
w.preprocess(preproc)
w

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Brazil'), ('price', 'France'), ('price', 'Germany'), ('price', 'Hungary'), ('price', 'Ukraine')]

In [6]:
w.train_df.tail(5)

price_trend price_seasonal price_residual price_trend  \
                 Brazil         Brazil         Brazil      France   
TIME_PERIOD                                                         
2021-11-01     0.743382       2.263731       0.204131    1.438160   
2021-12-01     0.714017      -0.224472      -0.377570    1.367648   
2022-01-01     0.681681       0.283744      -0.015838    1.292036   
2022-02-01     0.646431       0.448532      -0.034575    1.211759   
2022-03-01     0.608361       0.045353      -0.498172    1.127247   

            price_seasonal price_residual price_trend price_seasonal  \
                    France         France     Germany        Germany   
TIME_PERIOD                                                            
2021-11-01       -0.400818       0.065866    1.312617      -0.159450   
2021-12-01        0.555246      -0.739782    1.388919       0.007252   
2022-01-01       -0.178856       0.221264    1.465471       0.088056   
2022-02-01        0.485849      -0.426413    1.542429       0.062710   
2022-03-01        0.413221      -0.191965    1.619972       0.193871   

            price_residual price_trend  ... unemployment_total            \
                   Germany     Hungary  ...             Brazil    France   
TIME_PERIOD                             ...                                
2021-11-01       -0.168582    1.059052  ...           1.390065 -1.339159   
2021-12-01       -0.433435    1.029807  ...           1.390065 -1.339159   
2022-01-01       -0.000102    1.000852  ...           1.390065 -1.339159   
2022-02-01        0.116764    0.972248  ...           1.390065 -1.339159   
2022-03-01        0.232212    0.944159  ...           1.390065 -1.339159   

                                             price                      \
              Germany   Hungary  Ukraine    Brazil    France   Germany   
TIME_PERIOD                                                              
2021-11-01  -0.938878 -1.122596  1.39341  3.211244  1.103208  0.984585   
2021-12-01  -0.938878 -1.122596  1.39341  0.111976  1.183113  0.962737   
2022-01-01  -0.938878 -1.122596  1.39341  0.949587  1.334444  1.553425   
2022-02-01  -0.938878 -1.122596  1.39341  1.060388  1.271195  1.721903   
2022-03-01  -0.938878 -1.122596  1.39341  0.155542  1.348502  2.046054   

                                 
              Hungary   Ukraine  
TIME_PERIOD                      
2021-11-01   2.353540  0.980424  
2021-12-01   0.948267  1.669260  
2022-01-01   1.946410  1.251746  
2022-02-01   1.881186  1.750171  
2022-03-01   1.755316  1.869880  

[5 rows x 100 columns]

In [7]:
all(w.train_df == w.val_df)

True

In [8]:
w.test_df.head(5)

price_trend price_seasonal price_residual price_trend  \
                 Brazil         Brazil         Brazil      France   
TIME_PERIOD                                                         
2019-11-01    -1.061120       2.491796       0.554203   -0.813964   
2019-12-01    -0.946991       0.604478      -0.155318   -0.636971   
2020-01-01    -0.831852       0.296191      -0.024591   -0.459271   
2020-02-01    -0.715949       0.294615      -0.056375   -0.281182   
2020-03-01    -0.599513       0.460355      -0.290690   -0.103164   

            price_seasonal price_residual price_trend price_seasonal  \
                    France         France     Germany        Germany   
TIME_PERIOD                                                            
2019-11-01        0.395059       0.121637    0.381052      -0.073458   
2019-12-01        3.231146       0.302362    0.401443       0.803525   
2020-01-01        0.364914       0.178273    0.420685      -0.094930   
2020-02-01        0.734978      -0.350700    0.438861       0.401916   
2020-03-01        0.411530      -0.266848    0.456155       0.201118   

            price_residual price_trend  ... unemployment_total            \
                   Germany     Hungary  ...             Brazil    France   
TIME_PERIOD                             ...                                
2019-11-01       -0.130781    0.425039  ...           1.499367 -1.086280   
2019-12-01       -0.141944    0.490043  ...           1.564357 -1.125184   
2020-01-01       -0.102406    0.554623  ...           1.629347 -1.164089   
2020-02-01       -0.051392    0.618676  ...           1.609407 -1.178678   
2020-03-01        0.006598    0.682018  ...           1.589467 -1.193267   

                                              price                      \
              Germany   Hungary   Ukraine    Brazil    France   Germany   
TIME_PERIOD                                                               
2019-11-01  -0.862370 -1.099751  0.873988  1.984879 -0.297268  0.176812   
2019-12-01  -0.835368 -1.074127  0.972816 -0.497831  2.896538  1.063024   
2020-01-01  -0.808365 -1.048502  1.071644 -0.560252  0.083916  0.223349   
2020-02-01  -0.819241 -1.054677  1.098458 -0.477709  0.103096  0.789385   
2020-03-01  -0.830117 -1.060851  1.125272 -0.429849  0.041517  0.663871   

                                 
              Hungary   Ukraine  
TIME_PERIOD                      
2019-11-01   0.109866 -0.888247  
2019-12-01  -0.321027 -0.834174  
2020-01-01   2.754658 -0.815125  
2020-02-01   0.244221 -0.724015  
2020-03-01   0.155470 -0.503040  

[5 rows x 100 columns]

In [9]:
label_std = decomposition.Standardizer(mean=std.mean[w.label_columns], std=std.std[w.label_columns])
label_log = decomposition.Logger(label_indices=range(len(w.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
w.add_label_postprocess(postproc)

In [10]:
for example_inputs, example_labels in w.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 24, 95)
Labels shape (batch, time, features): (32, 6, 5)


In [11]:
def build_ARF(hp):
    lstm_units = hp.Int("lstm_units", min_value=32, max_value=512, step=32)
    lstm_layers = hp.Int("lstm_layers", min_value=0, max_value=10)
    prediction_units = hp.Int("prediction_units", min_value=32, max_value=512, step=32)
    prediction_layers = hp.Int("prediction_layers", min_value=0, max_value=10)
    feature_units = hp.Int("feature_units", min_value=32, max_value=512, step=32)
    feature_layers = hp.Int("feature_layers", min_value=0, max_value=10)
    
    heads = hp.Int("heads", min_value=1, max_value=16)
    dropout = hp.Float("dropout", min_value=0, max_value=1)
    key_dim = hp.Int('key_dim', min_value=16, max_value=128, step=16)

    model = models.AutoregressiveFeedback(out_steps=label_width, number_of_features=5, lstm_units=lstm_units, lstm_layers=lstm_layers,
                                   prediction_units=prediction_units, prediction_layers=prediction_layers, feature_units=feature_units,
                                   feature_layers=feature_layers, key_dim=key_dim, heads=heads, dropout=dropout)
    learning_rate = hp.Float("learning_rate", min_value=1e-7, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse', 
        metrics=['mae', 'mse', 'mape']
        )

    return model

In [12]:
def build_SS(hp):
    lstm_units = hp.Int("lstm_units", min_value=32, max_value=512, step=32)
    lstm_layers = hp.Int("lstm_layers", min_value=0, max_value=10)
    dense_units = hp.Int("dense_units", min_value=32, max_value=512, step=32)
    dense_layers = hp.Int("dense_layers", min_value=0, max_value=10)
    

    heads = hp.Int("heads", min_value=1, max_value=16)
    dropout = hp.Float("dropout", min_value=0, max_value=1)
    key_dim = hp.Int('key_dim', min_value=16, max_value=128, step=16)

    # m = EncoderDecoder(out_steps=label_width, number_of_features=5, units=units, blocks=blocks, heads=heads, dropout=dropout)
    model = models.SingleShot(out_steps=label_width, number_of_features=5, lstm_units=lstm_units, lstm_layers=lstm_layers, dense_units=dense_units, dense_layers=dense_layers, key_dim=key_dim, heads=heads, dropout=dropout)

    learning_rate = hp.Float("learning_rate", min_value=1e-7, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse', 
        metrics=['mae', 'mse', 'mape']
        )

    return model

In [13]:
def build_ED(hp):
    encoder_units = hp.Int("encoder_units", min_value=32, max_value=512, step=32)
    encoder_layers = hp.Int("encoder_layers", min_value=0, max_value=10)
    decoder_units = hp.Int("decoder_units", min_value=32, max_value=512, step=32)
    decoder_layers = hp.Int("decoder_layers", min_value=0, max_value=10)
    dense_units = hp.Int("dense_units", min_value=32, max_value=512, step=32)
    dense_layers = hp.Int("dense_layers", min_value=0, max_value=10)

    heads = hp.Int("heads", min_value=1, max_value=16)
    dropout = hp.Float("dropout", min_value=0, max_value=1)
    key_dim = hp.Int('key_dim', min_value=16, max_value=128, step=16)

    model = models.EncoderDecoder(out_steps=label_width, number_of_features=5, encoder_units=encoder_units, encoder_layers=encoder_layers,
                           decoder_units=decoder_units, decoder_layers=decoder_layers, dense_units=dense_units,
                           dense_layers=dense_layers, key_dim=key_dim, heads=heads, dropout=dropout)

    learning_rate = hp.Float("learning_rate", min_value=1e-7, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse', 
        metrics=['mae', 'mse', 'mape']
        )

    return model

In [14]:

tuner_arf = keras_tuner.Hyperband(
    hypermodel=build_ARF,
    objective="val_mse",
    max_epochs=200,
    factor=3,
    hyperband_iterations=1,
    executions_per_trial=5,
    seed=2023,
    max_retries_per_trial=10,
    max_consecutive_failed_trials=10,
    overwrite=False,
    directory=gonem_directory/'hp',
    project_name="ARF",
)

In [15]:
tuner_ss = keras_tuner.Hyperband(
    hypermodel=build_SS,
    objective="val_mse",
    max_epochs=200,
    factor=3,
    hyperband_iterations=1,
    executions_per_trial=5,
    seed=2023,
    max_retries_per_trial=10,
    max_consecutive_failed_trials=10,
    overwrite=False,
    directory=gonem_directory/'hp',
    project_name="SS",
)

In [16]:
tuner_ed = keras_tuner.Hyperband(
    hypermodel=build_ED,
    objective="val_mse",
    max_epochs=200,
    factor=3,
    hyperband_iterations=1,
    executions_per_trial=5,
    seed=2023,
    max_retries_per_trial=10,
    max_consecutive_failed_trials=10,
    overwrite=False,
    directory=gonem_directory/'hp',
    project_name="ED",
)

In [17]:
tuner_arf.search(w.train, validation_data=w.val, verbose=2)

Trial 138 Complete [00h 00m 00s]

Best val_mse So Far: 0.45112399458885194
Total elapsed time: 07h 36m 40s

Search: Running Trial #139

Value             |Best Value So Far |Hyperparameter
352               |192               |lstm_units
4                 |1                 |lstm_layers
416               |512               |prediction_units
8                 |1                 |prediction_layers
288               |32                |feature_units
9                 |7                 |feature_layers
15                |15                |heads
0.17531           |0.069788          |dropout
48                |80                |key_dim
1.343e-06         |0.00055806        |learning_rate
8                 |8                 |tuner/epochs
0                 |3                 |tuner/initial_epoch
3                 |4                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/8
6/6 - 23s - loss: 0.9394 - mae: 0.7139 - mse: 0.9394 - mape: 143.1283 - val_loss: 0.92

In [ ]:
tuner_ss.search(w.train, epochs=1000, validation_data=w.val, verbose=2)

In [ ]:
tuner_ed.search(w.train, epochs=1000, validation_data=w.val, verbose=2)

In [ ]:
m_arf = tuner_arf.get_best_models(num_models=1)[0]

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=gonem_directory/'best_models', monitor='val_loss', save_best_only=True)